# 自己再編成リスト

In [145]:
class MyList:
    """
    自己再編成は行わない
    """
    def __init__(self, kv_list):
        n = len(kv_list)
        self.data = []
        self.i_head = 0
        self.data.append([kv_list[0][0], kv_list[0][1], None, 1])
        for i in range(1, n-1):
            self.data.append([kv_list[i][0], kv_list[i][1], i-1, i+1])
        self.data.append([kv_list[n-1][0], kv_list[n-1][1], n-2, None])
    
    def get_result_elem(self, key):
        i_next = self.i_head
        while i_next is not None:
            i_now = i_next
            k, v, i_prev, i_next = self.data[i_now]
            if k == key:
                return k, v, i_prev, i_next, i_now
        raise Exception('key {} not found'.format(key))

    def search(self, key):
        k, v, i_prev, i_next, i_target = self.get_result_elem(key)
        return v

class SelfOrganizingMyList1(MyList):
    """
    検索されたデータを先頭に挿入する
    """
    def search(self, key):
        k, v, i_prev, i_next, i_target = self.get_result_elem(key)
        if i_target != self.i_head:
            self.data[i_target][2], self.data[i_target][3] = None, self.i_head
            self.data[self.i_head][2] = i_target
            self.data[i_prev][3] = i_next
            if i_next is not None:
                self.data[i_next][2] = i_prev
            self.i_head = i_target
        return v

class SelfOrganizingMyList2(MyList):
    """
    検索されたデータを1つ前と交換する
    """
    def search(self, key):
        k, v, i_prev, i_next, i_target = self.get_result_elem(key)
        if i_target != self.i_head:
            self.data[i_target][0], self.data[i_target][1] = self.data[i_prev][0], self.data[i_prev][1]
            self.data[i_prev][0], self.data[i_prev][1] = k, v
        return v

In [146]:
l = [['a', 100], ['b', 200], ['c', 300], ['d', 400], ['e', 500]]

def func(list_impl):
    print(list_impl.data)
    for k in ['a', 'b', 'd', 'd', 'c', 'a', 'e', 'c', 'e', 'b']:
        v = list_impl.search(k)
        print('k={}, v={}, list.data={}, list.i_head={}'.format(k, v, list_impl.data, list_impl.i_head))

In [147]:
func(MyList(l))

[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]]
k=a, v=100, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=b, v=200, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=d, v=400, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=d, v=400, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=c, v=300, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=a, v=100, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=e, v=500, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], [

In [148]:
func(SelfOrganizingMyList1(l))

[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]]
k=a, v=100, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=b, v=200, list.data=[['a', 100, 1, 2], ['b', 200, None, 0], ['c', 300, 0, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=1
k=d, v=400, list.data=[['a', 100, 1, 2], ['b', 200, 3, 0], ['c', 300, 0, 4], ['d', 400, None, 1], ['e', 500, 2, None]], list.i_head=3
k=d, v=400, list.data=[['a', 100, 1, 2], ['b', 200, 3, 0], ['c', 300, 0, 4], ['d', 400, None, 1], ['e', 500, 2, None]], list.i_head=3
k=c, v=300, list.data=[['a', 100, 1, 4], ['b', 200, 3, 0], ['c', 300, None, 3], ['d', 400, 2, 1], ['e', 500, 0, None]], list.i_head=2
k=a, v=100, list.data=[['a', 100, None, 2], ['b', 200, 3, 4], ['c', 300, 0, 3], ['d', 400, 2, 1], ['e', 500, 1, None]], list.i_head=0
k=e, v=500, list.data=[['a', 100, 4, 2], ['b', 200, 3, None], ['c', 300, 0, 3], ['d', 400, 2, 1], [

In [149]:
func(SelfOrganizingMyList2(l))

[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]]
k=a, v=100, list.data=[['a', 100, None, 1], ['b', 200, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=b, v=200, list.data=[['b', 200, None, 1], ['a', 100, 0, 2], ['c', 300, 1, 3], ['d', 400, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=d, v=400, list.data=[['b', 200, None, 1], ['a', 100, 0, 2], ['d', 400, 1, 3], ['c', 300, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=d, v=400, list.data=[['b', 200, None, 1], ['d', 400, 0, 2], ['a', 100, 1, 3], ['c', 300, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=c, v=300, list.data=[['b', 200, None, 1], ['d', 400, 0, 2], ['c', 300, 1, 3], ['a', 100, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=a, v=100, list.data=[['b', 200, None, 1], ['d', 400, 0, 2], ['a', 100, 1, 3], ['c', 300, 2, 4], ['e', 500, 3, None]], list.i_head=0
k=e, v=500, list.data=[['b', 200, None, 1], ['d', 400, 0, 2], ['a', 100, 1, 3], ['e', 500, 2, 4], [